In [1]:
import pandas as pd
from Levenshtein import distance as levenshtein_distance
pd.set_option('display.max_columns', None)

In [2]:
data = pd.read_csv(r'C:\Users\Mufct\Desktop\Minor DH and SA\DH in Practice\Metadata-final-NHA-tsv.csv')
data = data[data['Locaties-Locatie-Straat'].isna() != True]

length_of_data = len(data)
data = data.reset_index(drop = True)
iteration = 0
distances = pd.DataFrame()
for street in data['Locaties-Locatie-Straat']:
    leven_distance = levenshtein_distance(street, data.loc[iteration, 'located on street'])
    distances.loc[iteration,'Street'] = data.loc[iteration,'Locaties-Locatie-Straat']
    distances.loc[iteration, 'Proposed Street'] = data.loc[iteration,'located on street']
    distances.loc[iteration,'Levenshtein Distance'] = leven_distance
    iteration += 1

In [3]:
above_distance_of_3 = 0
for i in range(0,len(distances)):
    if distances.loc[i,'Levenshtein Distance'] > 3.0:
        above_distance_of_3 += 1

In [4]:
in_address = 0
not_in = 0
didnotmatch = pd.DataFrame()
for j in range(0,len(distances)):
    if distances.loc[j,'Street'].lower() in distances.loc[j,'Proposed Street'].lower():
        in_address += 1
    else:
        didnotmatch.loc[not_in,'Street'] = distances.loc[j,'Street']
        didnotmatch.loc[not_in,'Proposed Street'] = distances.loc[j,'Proposed Street']
        not_in += 1

In [ ]:
geolocator = Nominatim(user_agent="http")
for i in range(0,len(didnotmatch)): 
    street = didnotmatch.loc[i,'Street'] + ', Haarlem'
    location = geolocator.geocode(street)
    if location is None:
        didnotmatch.loc[i,'Street Coordinates'] = 0
    else:
        didnotmatch.loc[i,'Street Coordinates'] = '(' + str(location.latitude) + ', ' + str(location.longitude) + ')'
    proposed_street = didnotmatch.loc[i,'Proposed Street'] + ', Haarlem'
    proposed_location = geolocator.geocode(proposed_street)
    if proposed_location is None:
        didnotmatch.loc[i,'Proposed Street Coordinates'] = 0
    else:
        didnotmatch.loc[i,'Proposed Street Coordinates'] = '(' + str(proposed_location.latitude) + ', ' + str(proposed_location.longitude) + ')'

In [ ]:
didnotmatch = didnotmatch[didnotmatch['Street Coordinates'] != 0]
didnotmatch = didnotmatch[didnotmatch['Proposed Street Coordinates'] != 0]
didnotmatch = didnotmatch.reset_index(drop = True)


In [ ]:
import geopy.distance
for j in range(0,len(didnotmatch)):
    coordinates = didnotmatch.loc[j,'Street Coordinates'].replace('(','').replace(')','').split(',')
    coords_1 = (coordinates[0],coordinates[1].strip())
    proposed_coordinates = didnotmatch.loc[j,'Proposed Street Coordinates'].replace('(','').replace(')','').split(',')
    coords_2 = (proposed_coordinates[0],proposed_coordinates[1].strip())
    didnotmatch.loc[j,'Distance between Locations'] = geopy.distance.geodesic(coords_1, coords_2).km

In [84]:
total = 560
less_than_100m = 0
less_than_250m = 0
less_than_500m = 0
less_than_1km = 0
for k in range(0,len(didnotmatch)):
    if didnotmatch.loc[k,'Distance between Locations'] < 0.1:
        less_than_100m += 1
    if didnotmatch.loc[k,'Distance between Locations'] < 0.25:
        less_than_250m += 1
    if didnotmatch.loc[k,'Distance between Locations'] < 0.5:
        less_than_500m += 1
    if didnotmatch.loc[k,'Distance between Locations'] < 1:
        less_than_1km += 1
less_than_100m,less_than_250m,less_than_500m, less_than_1km, total

(168, 256, 358, 457, 560)